# Python으로 웹 스크래퍼 만들기 2

사이트 운영자들은 웹 스크래핑 매우 싫어한다.

그래서 봇 인증 프로그램 등을 만들어 작업을 까다롭게 조치를 해두는 경우가 있다. ( 이용약관에도 기입해 두기도 한다. )

민감한 사항이 때문에 스크래핑한 내용을 상업적으로 사용할 경우 법적다툼이 벌어질 가능성도 있다.

교육 목적 이외에 상업적인 서비스를 만들 생각이라면 참고하자.


- URL Formatting

- Requests

- [HTTP Status Codes](https://developer.mozilla.org/ko/docs/Web/HTTP/Status)

- dictionary

- .startswith

- f"{}"

- find_all : list

- find : v

---

## ~ 5.9

In [ ]:
from requests import get 

from bs4 import BeautifulSoup


base_url = "https://weworkremotely.com/remote-jobs/search?utf8=✓&term="

search_term = "java"

response = get(f"{base_url}{search_term}")

    
if response.status_code != 200:
    print("Can't request website")
else:
    soup = BeautifulSoup(response.text,"html.parser")
    
    # print(soup.find_all('title'))

    jobs = soup.find_all('section', class_="jobs")
    # print(soup.find_all('section', class_="jobs"))    
    # print(len(jobs))
    
    results = []
    
    for job_section in jobs:
    
        # print(job_section.find_all('li')) 
        job_posts = job_section.find_all('li')
        
        # 리스트 컴프리헨션으로 view-all 클래스 제외 (버튼)
        job_posts = [post for post in job_section.find_all('li') if 'view-all' not in post['class']]

        for post in job_posts:
            
            
            anchors = post.find_all('a') # list
            
            anchor = anchors[1] # dictionary in list
            
            link = anchor['href'] # value in dictionary
            
            # anchor('a') 내 span tag (class = company)추출, list
            company, position, region = anchor.find_all('span',class_='company')
            
            title = anchor.find('span', class_='title')
            
            # html tag 제외 후, dictionary 데이터 생성 후, list append
            data_result = {
                'title': title.string,
                'company' : company.string,
                'position' : position.string,
                'region' : region.string,
                'link' : f'https://weworkremotely.com{link}'
                }
            
            results.append(data_result)
        
    
    for result in results:
        print(result)

            

---

## Refactoring

- search_term 을 parameter 로 받고, dict 데이터로 results 를 return 하는 함수 생성


In [ ]:
from requests import get 
from bs4 import BeautifulSoup

# extract 폴더, wwr.py 파일, extract_jobs 함수 import

from extract.wwr import extract_jobs

jobs = extract_jobs('python')
print(jobs)

---

## Request 응답 거절

- Request로 접속, response code : 403


- 403 Forbidden

클라이언트는 콘텐츠에 접근할 권리를 가지고 있지 않습니다. 

예를들어 그들은 미승인이어서 서버는 거절을 위한 적절한 응답을 보냅니다. 401과 다른 점은 서버가 클라이언트가 누구인지 알고 있습니다.


In [ ]:
from requests import get
from bs4 import BeautifulSoup


base_url = "https://kr.indeed.com/jobs?q="
search_term = "python"

response = get(f"{base_url}{search_term}")

if response.status_code != 200:
    print("Cant request website")
else:
    print(response.text)

In [ ]:
print(response.status_code)

---

## Selenium

- https://www.selenium.dev/

- requests get 으로 403 forbidden, Selenium 을 이용하여 해결
---

BeautifulSoup 한계

바로, "자바스크립트로 동적으로 생성된 정보는 가져올 수 없다!"입니다.

자바스크립트가 발전을 하면서, Ajax(비동기 통신) 형태로 서버와 데이터를 주고 받아 화면에 뿌려주는 사이트가 많아 졌습니다. 

이러한 형식으로 데이터를 주고 받으면 url 변경이나 새로고침 없이 데이터를 가져오게 됩니다.

아마도 스크래핑을 시도하다가 데이터를 가져와야 정상인데, 아무것도 가져오지 못하는 현상을 마주친적이 있을 거에요. 

대부분의 경우가 자바스크립트로 HTML을 만들어서 그렇습니다.

Selenium 라이브러리를 사용하는 이유는 다음과 같습니다.

    1. 자바스크립트가 동적으로 만든 데이터를 크롤링 하기 위해

    2. 사이트의 다양한 HTML 요소에 클릭, 키보드 입력 등 이벤트를 주기 위해

Selenium을 잘 활용하면, 평소에 반복적으로 하고 있는 웹상의 업무를 자동화할 수도 있습니다.

    1. 자동으로 로그인하기

    2. 메일보내기 자동화

    3. 블로그 이웃새글 자동좋아요 누르기

    4. 인스타그램 자동으로 좋아요, 댓글 작성하기

    5. 등등 정말 많은 다양한 일




In [ ]:
# indeed 403 fix

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

# for Replit option

# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

# response = get(f"{base_url}{search_term}")
browser.get(f"{base_url}?q={search_term}")

print(browser.page_source)

In [ ]:
type(browser)

---

<img src = '2.png' height ='500' width = '500'>



## Recursive


1. find 로 `ul` 찾음


2. find_all 로 `ul` 안에 li 모두 찾음 (li 안에 li) 


- `ul` 바로 안에 `li` 만을 찾길 원함. 
-  `li` 안에 `li` 제외 
- 1 level 만 들어감 깊게 들어가는거 방지

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

browser.get(f"{base_url}?q={search_term}")

# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

job_list = soup.find("ul", class_="jobsearch-ResultsList")

jobs = job_list.find_all('li', recursive=False)

print(len(jobs))
print()

for job in jobs:
    print(job)

---

## None

- 무언가 '없읍'

- false 랑 다름

- html 데이터 -> ul -> li (recursive=False)

- 이 후 필터링 으로 "div", class_="mosaic-zone" 데이터 처리 



<img src = '3.png' height ='500' width = '500'>

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

browser.get(f"{base_url}?q={search_term}")

# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

job_list = soup.find("ul", class_="jobsearch-ResultsList")

jobs = job_list.find_all('li', recursive=False)


# li 태그 중 class 네임에 따라 출력,
# find, none 논리로 출력 

for job in jobs:
    
    zone = job.find("div", class_="mosaic-zone")
    
    if zone == None:
        print("job li")
    else:
        print("mosaic li")

---

## Select

- select_one > dict 형으로 한번에 

- find_all('') > 태그 를 리스트화 


- https://nomadcoders.co/python-for-beginners/lectures/3801

In [45]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

browser.get(f"{base_url}?q={search_term}")

# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

job_list = soup.find("ul", class_="jobsearch-ResultsList")

jobs = job_list.find_all('li', recursive=False)


# li 태그 중 class 네임에 따라 출력,
# find, none 논리로 출력 

results = []

for job in jobs:
    
    zone = job.find("div", class_="mosaic-zone")
    
    if zone == None:
        
        # anchor = job.find_all('a') # list

        anchor = job.select_one('h2 a') # dict
        
        title = anchor['aria-label']
        link = anchor['href']
        
        company = job.find('span', class_='companyName')
        location = job.find('div', class_='companyLocation')
    
        # dict 데이터 형
        job_data = {
            'position': title,
            'link': f"https://kr.indeed.com{link}",
            'company': company.string,
            'location': location.string
        }
        
        # 결과 리스트 append
        results.append(job_data)
    
    else:
        print("mosaic li")
    
for result in results:
    print(result)
    print()

mosaic li
mosaic li
mosaic li
{'position': 'Financial, Regulatory and Risk Reporting Speciallist의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=4e842304541b3a53&fccid=f1d8e147024abb3f&vjs=3', 'company': 'Deutsche Bank', 'location': '서울'}

{'position': '인공지능(AI), 인공지능(AI) 외 신입의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=c5112e132f7f1e05&fccid=03ad1f6903097dae&vjs=3', 'company': '사단법인 한국서비스산업진흥원', 'location': '서울 종로구'}

{'position': '백엔드 엔지니어 (Python) 정규직의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=79d696388d32d3ff&fccid=dd6f5fd8e8c02e7e&vjs=3', 'company': '데이터비', 'location': '서울 마포구'}

{'position': '그룹공동 마이데이터 분석 플랫폼(데이터분석가)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=25b3c3e6a66bd77c&fccid=fe227fbd2afb9580&vjs=3', 'company': 'KB데이타시스템', 'location': '서울 마포구'}

{'position': 'Python Backend Engineer의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b2fc2142a72a2007&fccid=6e36b32060404d12&vjs=3', 'company': '마인드에이아이', 'location': '서울 중구'}

{'position': '

In [ ]:
print(title)

In [ ]:
print(link)

In [ ]:
print(company)

In [ ]:
print(location)

---

## pages


- 페이지가 1개 

- 페이지가 여러개, 이동 버튼 없음

- 페이지가 여러개, 이동 버튼 있음 (6개이상 존재)


1. 태그랑, class 네임 둘다 변경 `nav`, `div`, `css-jbuxu0 ecydgvn0`


2. `pagination = soup.find('nav', class_='css-jbuxu0 ecydgvn0')`   시 일단 모두 생성됨. none 으로 return x 일단 생성 o


3. `pages = pagination.find_all('div', recursive=False)` 있으면 리스트화, 없으면 [] 빈 리스트 
    - <class 'bs4.element.ResultSet'> 
    
 



In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options



def get_page_count(keyword):
    
    options = Options()
    browser = webdriver.Chrome(options=options)

    base_url = "https://kr.indeed.com/jobs"
    browser.get(f"{base_url}?q={keyword}")
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    pagination = soup.find('nav', class_='css-jbuxu0 ecydgvn0')    
    
    pages = pagination.find_all('div', recursive=False)
    length = len(pages)
    
    # 5를 초과하는 6 이상 부터는 처리 하지 않을 예정
    if length == 0:
        return 1
    elif length > 5:
        return 5
    else: 
        return length

In [5]:
get_page_count('react')


5

In [29]:
get_page_count('nest')


1

## Refactoring - 2

- 함수화
- pages 추가


In [43]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options




def extract_indeed_jobs(keyword):
    
    pages = get_page_count(keyword)
    
    for page in range(pages):
        
        options = Options()

        browser = webdriver.Chrome(options=options)
        
        base_url = "https://kr.indeed.com/jobs"
        search_term = f'{keyword}'

        browser.get(f"{base_url}?q={search_term}")

        # --- bs4 처리 

        soup = BeautifulSoup(browser.page_source, "html.parser")

        job_list = soup.find("ul", class_="jobsearch-ResultsList")

        jobs = job_list.find_all('li', recursive=False)


        # li 태그 중 class 네임에 따라 출력,
        # find, none 논리로 출력 

        results = []

        for job in jobs:

            zone = job.find("div", class_="mosaic-zone")

            if zone == None:

                # anchor = job.find_all('a') # list

                anchor = job.select_one('h2 a') # dict

                title = anchor['aria-label']
                link = anchor['href']

                company = job.find('span', class_='companyName')
                location = job.find('div', class_='companyLocation')

                # dict 데이터 형
                job_data = {
                    'position': title,
                    'link': f"https://kr.indeed.com{link}",
                    'company': company.string,
                    'location': location.string
                }

                # 결과 리스트 append
                results.append(job_data)

        for result in results:
            print(result)
            print()

In [44]:
# 1번 페이지 5번 반복 

extract_indeed_jobs('python')

mosaic li
mosaic li
mosaic li
{'position': 'Financial, Regulatory and Risk Reporting Speciallist의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=4e842304541b3a53&fccid=f1d8e147024abb3f&vjs=3', 'company': 'Deutsche Bank', 'location': '서울'}

{'position': '인공지능(AI), 인공지능(AI) 외 신입의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=c5112e132f7f1e05&fccid=03ad1f6903097dae&vjs=3', 'company': '사단법인 한국서비스산업진흥원', 'location': '서울 종로구'}

{'position': '백엔드 엔지니어 (Python) 정규직의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=79d696388d32d3ff&fccid=dd6f5fd8e8c02e7e&vjs=3', 'company': '데이터비', 'location': '서울 마포구'}

{'position': '그룹공동 마이데이터 분석 플랫폼(데이터분석가)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=25b3c3e6a66bd77c&fccid=fe227fbd2afb9580&vjs=3', 'company': 'KB데이타시스템', 'location': '서울 마포구'}

{'position': 'Python Backend Engineer의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b2fc2142a72a2007&fccid=6e36b32060404d12&vjs=3', 'company': '마인드에이아이', 'location': '서울 중구'}

{'position': '

mosaic li
mosaic li
mosaic li
{'position': 'Financial, Regulatory and Risk Reporting Speciallist의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=4e842304541b3a53&fccid=f1d8e147024abb3f&vjs=3', 'company': 'Deutsche Bank', 'location': '서울'}

{'position': '인공지능(AI), 인공지능(AI) 외 신입의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=c5112e132f7f1e05&fccid=03ad1f6903097dae&vjs=3', 'company': '사단법인 한국서비스산업진흥원', 'location': '서울 종로구'}

{'position': '백엔드 엔지니어 (Python) 정규직의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=79d696388d32d3ff&fccid=dd6f5fd8e8c02e7e&vjs=3', 'company': '데이터비', 'location': '서울 마포구'}

{'position': '그룹공동 마이데이터 분석 플랫폼(데이터분석가)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=25b3c3e6a66bd77c&fccid=fe227fbd2afb9580&vjs=3', 'company': 'KB데이타시스템', 'location': '서울 마포구'}

{'position': 'Python Backend Engineer의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b2fc2142a72a2007&fccid=6e36b32060404d12&vjs=3', 'company': '마인드에이아이', 'location': '서울 중구'}

{'position': '

---

`options = Options()`
`browser = webdriver.Chrome(options=options)`

를 함수 내에서 실행시키게 해야함.

- 켜져있는 상태로, 한번 더 켜지면, 봇 의심

- 함수 내에서 오픈하게 되면, 실행 후, 꺼지고, 다시 실행

- 오픈, 카운트, 클로즈

- 1~5. 오픈, 출력, 클로즈



---

## Refactoring -3

- 1 ~ 5 번페이지 start 처리

- page 에 따라 start 부분 처리 x(0), 10, 20

- start를 0처리 하면, start 없이 요청 하는것과 같은 결과가 나옴

- result 빼서 처리

In [48]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def extract_indeed_jobs(keyword):
    
    pages = get_page_count(keyword)
    
    results = []
  
    for page in range(pages):
        
        options = Options()

        browser = webdriver.Chrome(options=options)
        
        base_url = "https://kr.indeed.com/jobs"
        search_term = f'{keyword}'

        browser.get(f"{base_url}?q={search_term}&start={page*10}")

        # --- bs4 처리 

        soup = BeautifulSoup(browser.page_source, "html.parser")

        job_list = soup.find("ul", class_="jobsearch-ResultsList")

        jobs = job_list.find_all('li', recursive=False)


        # li 태그 중 class 네임에 따라 출력,
        # find, none 논리로 출력 


        for job in jobs:

            zone = job.find("div", class_="mosaic-zone")

            if zone == None:

                # anchor = job.find_all('a') # list

                anchor = job.select_one('h2 a') # dict

                title = anchor['aria-label']
                link = anchor['href']

                company = job.find('span', class_='companyName')
                location = job.find('div', class_='companyLocation')

                # dict 데이터 형
                job_data = {
                    'position': title,
                    'link': f"https://kr.indeed.com{link}",
                    'company': company.string,
                    'location': location.string
                }

                # 결과 리스트 append
                results.append(job_data)

    
    for result in results:
        print(result)
        print()

In [49]:
extract_indeed_jobs('python')

{'position': 'Financial, Regulatory and Risk Reporting Speciallist의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=4e842304541b3a53&fccid=f1d8e147024abb3f&vjs=3', 'company': 'Deutsche Bank', 'location': '서울'}

{'position': '인공지능(AI), 인공지능(AI) 외 신입의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=c5112e132f7f1e05&fccid=03ad1f6903097dae&vjs=3', 'company': '사단법인 한국서비스산업진흥원', 'location': '서울 종로구'}

{'position': '백엔드 엔지니어 (Python) 정규직의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=79d696388d32d3ff&fccid=dd6f5fd8e8c02e7e&vjs=3', 'company': '데이터비', 'location': '서울 마포구'}

{'position': '그룹공동 마이데이터 분석 플랫폼(데이터분석가)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=25b3c3e6a66bd77c&fccid=fe227fbd2afb9580&vjs=3', 'company': 'KB데이타시스템', 'location': '서울 마포구'}

{'position': 'Python Backend Engineer의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b2fc2142a72a2007&fccid=6e36b32060404d12&vjs=3', 'company': '마인드에이아이', 'location': '서울 중구'}

{'position': '[비에이코리아 주식회사] 데이터분석가(Business 

---

- 완전 함수로 변환

- return 리스트

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def extract_indeed_jobs(keyword):
    
    pages = get_page_count(keyword)
    
    results = []
  
    for page in range(pages):
        
        options = Options()

        browser = webdriver.Chrome(options=options)
        
        base_url = "https://kr.indeed.com/jobs"
        search_term = f'{keyword}'
        final_url = f"{base_url}?q={search_term}&start={page*10}"
        
        print(f'Request url : {final_url}')
        browser.get(final_url)

        # --- bs4 처리 

        soup = BeautifulSoup(browser.page_source, "html.parser")

        job_list = soup.find("ul", class_="jobsearch-ResultsList")

        jobs = job_list.find_all('li', recursive=False)


        # li 태그 중 class 네임에 따라 출력,
        # find, none 논리로 출력 


        for job in jobs:

            zone = job.find("div", class_="mosaic-zone")

            if zone == None:

                # anchor = job.find_all('a') # list

                anchor = job.select_one('h2 a') # dict

                title = anchor['aria-label']
                link = anchor['href']

                company = job.find('span', class_='companyName')
                location = job.find('div', class_='companyLocation')

                # dict 데이터 형
                job_data = {
                    'position': title,
                    'link': f"https://kr.indeed.com{link}",
                    'company': company.string,
                    'location': location.string
                }

                # 결과 리스트 append
                results.append(job_data)

    
    return results

In [2]:
jobs = extract_indeed_jobs('python')
print(jobs)

NameError: name 'get_page_count' is not defined

In [55]:
print(len(jobs))

75


- 첫번째 페이지로 이동후, 그 이후 페이지가 몇개 있는지 확인, 제약사항(5) 후 처리
